In [57]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
sns.set(style="darkgrid")
import sys
sys.path.append('../../src')
sys.path.append('../../src/Vrama')
from Vrama import Eye
import time
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import bokeh.sampledata
# bokeh.sampledata.download()
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.palettes import Spectral4, all_palettes
from bokeh.models import ColumnDataSource, Whisker
import math
import statistics
output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [ ]:
#p.vbar(x=fruits, top=counts, width=0.9)

# class bar_scores:
#         def __init__(self,x,top,width):
#             self.x = x
#             self.top = top
#             self.width = width
            
# data_dict = {   'subjectID': subjectID,
#                 'dfAll': dfAll,
#                 'df_scores_all':df_scores_all,
#                 'df_scores_on':df_scores_on,
#                 'df_scores_off':df_scores_off           
#                     }

In [58]:
resultsList = []
subjList = ['19202','19205','19201']
for i in subjList:
    resultsList.append(Eye.GetEyeScores(i))

In [59]:
object_type = 'View_duration_distractor(%)'
colors = ["red", "olive", "darkred", "goldenrod", "skyblue", "orange", "salmon"]
task_list = ['Math','VerbalStroop','AX-CPT']
for result in resultsList:
    x_labels = ['distractor on','distractor off']
    x_inds = list(range(1,len(x_labels)+1))
    mean_l = pd.Series([result['df_scores_on'][object_type].mean(),result['df_scores_off'][object_type].mean()])
    std_l = pd.Series([result['df_scores_on'][object_type].std(),result['df_scores_off'][object_type].std()])
    ste_l = std_l/np.sqrt(len(result['df_scores_on'][object_type]))

    source_error = ColumnDataSource(data=dict(base=x_inds, lower=mean_l - ste_l, upper=mean_l + ste_l))
    width = .5
    
    p = figure(x_range= [min(x_inds)-.5,max(x_inds)+.5], plot_height=350, title=result['subjectID'] +' : Task blocks by distraction On/Off')

    #p.vbar(x=x, top=top, width=width)
#     p.xgrid.grid_line_color = None
#     p.y_range.start = 0
    p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

    for i, score in enumerate(list(result['df_scores_on'][object_type])):
        task = result['df_scores_on']['Task_name'].iloc[i]
        color_i = task_list.index(task)
        color = colors[color_i]
        p.circle(x=x_inds[0], y=score, color=color,size=5,legend=task)
    
    for i, score in enumerate(list(result['df_scores_off'][object_type])):
        task = result['df_scores_on']['Task_name'].iloc[i]
        color_i = task_list.index(task)
        color = colors[color_i]
        p.circle(x=x_inds[1], y=score, color=color,size=5,legend=task)
    
    p.yaxis.axis_label = "%"
    p.axis.minor_tick_in = 0
    p.axis.minor_tick_out = 0
    p.yaxis.major_tick_in = 0
    p.xaxis.major_tick_in = 0
    p.xaxis.major_tick_out = 3
    p.xaxis.ticker = [1,2]
    p.xaxis.major_label_overrides = {1: x_labels[0], 2: x_labels[1]}
    p.axis.major_label_text_font_size = "10pt"
    show(p)
    print(result['subjectID'])
    print(stats.ttest_rel(result['df_scores_on'][object_type],result['df_scores_off'][object_type])) 

19202
Ttest_relResult(statistic=1.7448832185705643, pvalue=0.12451898959684793)


19205
Ttest_relResult(statistic=3.2002393793198145, pvalue=0.015060802543077916)


19201
Ttest_relResult(statistic=4.402630661792442, pvalue=0.0022787288021962277)


In [63]:
object_type = 'View_duration_distractor(%)'
colors = ["goldenrod", "skyblue", "orange", "salmon"]
x_labels = ['distractor on','distractor off']
x_inds = list(range(1,len(x_labels)+1))
mean_on_l = []
mean_off_l = []
subject_l = []
for result in resultsList:   
    subject_l.append(result['subjectID'])
    mean_on_l.append(result['df_scores_on'][object_type].mean())
    mean_off_l.append(result['df_scores_off'][object_type].mean())


In [64]:
print(stats.ttest_rel(mean_on_l,mean_off_l))

Ttest_relResult(statistic=1.9831655565299877, pvalue=0.18581370635990246)


In [2]:
mean_l = [statistics.mean(mean_on_l),statistics.mean(mean_off_l)]
std_l = [statistics.stdev(mean_on_l),statistics.stdev(mean_off_l)]
ste_l = std_l/np.sqrt(len(result['df_scores_on'][object_type]))

source_error = ColumnDataSource(data=dict(base=x_inds, lower=mean_l - ste_l, upper=mean_l + ste_l))
width = .5

p = figure(x_range= [min(x_inds)-.5,max(x_inds)+.5], plot_height=350, title='All subjects: Task blocks by distraction On/Off')
p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower",line_width=3)
)
for i, score in enumerate(mean_on_l):
        subject = subject_l[i]
        color = colors[i]
        p.circle(x=x_inds[0], y=score, color=color,size=10,legend=subject)
    
for i, score in enumerate(mean_off_l):
        subject = subject_l[i]
        color = colors[i]
        p.circle(x=x_inds[1], y=score, color=color,size=10,legend=subject)
        
p.yaxis.axis_label = "mean %"
p.axis.minor_tick_in = 0
p.axis.minor_tick_out = 0
p.yaxis.major_tick_in = 0
p.xaxis.major_tick_in = 0
p.xaxis.major_tick_out = 3
p.xaxis.ticker = [1,2]
p.xaxis.major_label_overrides = {1: x_labels[0], 2: x_labels[1]}
p.axis.major_label_text_font_size = "10pt"
show(p)

NameError: name 'statistics' is not defined